In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

# DATA SIMETRIK

In [ ]:
allfiles_sim = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\PROVISIONES\2021\07. JULIO\DATA SIMETRIK - ADQ NO CLIP\*.csv', recursive=True)

In [ ]:
df_all_files_sim = pd.concat([pd.read_csv(f,dtype={'TRANSACTION_REFERENCE':object,'COD_AUTORIZACION':object,'BIN':object
                                                  ,'ULTIMOS_4':object,'REFERENCIA_PAYWORKS':object,'MONTO':float})
                              .assign(file_name_tableros=os.path.basename(f)) for f in allfiles_sim])
df_all_files_sim.columns = df_all_files_sim.columns.str.strip().str.lower()
df_all_files_sim.reset_index(inplace=True)
df_all_files_sim.loc[pd.notnull(df_all_files_sim['transaction_reference']),'order_id'] = df_all_files_sim['transaction_reference']
df_all_files_sim.loc[pd.notnull(df_all_files_sim['referencia_payworks']),'order_id'] = df_all_files_sim['referencia_payworks']
df_all_files_sim.loc[pd.notnull(df_all_files_sim['fecha_deposito_banorte']),'fecha_deposito'] = df_all_files_sim['fecha_deposito_banorte']
df_all_files_sim.loc[pd.notnull(df_all_files_sim['fecha_deposito_amex']),'fecha_deposito'] = df_all_files_sim['fecha_deposito_amex']
df_all_files_sim.loc[pd.notnull(df_all_files_sim['fecha_trx_amex']),'fecha_transaccion'] = df_all_files_sim['fecha_trx_amex']
df_all_files_sim.loc[pd.notnull(df_all_files_sim['fecha_trx_banorte']),'fecha_transaccion'] = df_all_files_sim['fecha_trx_banorte']
df_all_files_sim['order_id'] = df_all_files_sim['order_id'].str.upper()
df_all_files_sim['order_id'] = df_all_files_sim['order_id'].str.strip()
df_all_files_sim['order_id'].fillna('Sin order_id simetrik',inplace=True)
df_all_files_sim.head()

In [ ]:
df_all_files_sim.shape

In [ ]:
df_all_files_sim.columns

In [ ]:
df_all_files_sim_final = df_all_files_sim[['nombre_tablero','fecha_transaccion','fecha_deposito','order_id'
                                         ,'cod_autorizacion','bin','ultimos_4','msi','id_comercio','caso_uso','monto'
                                           ,'file_name_tableros']]
df_all_files_sim_final.head()

In [ ]:
df_all_files_sim_final.shape

In [ ]:
# df_all_files_sim_final.to_excel('DATA QUERY ANC.xlsx')

In [ ]:
df_all_files_sim_final_amex = df_all_files_sim_final[df_all_files_sim_final['file_name_tableros'].str.contains('AMEX')]
df_all_files_sim_final_grrcn = df_all_files_sim_final[df_all_files_sim_final['file_name_tableros'].str.contains('GRRCN')]
df_all_files_sim_final_bnte = df_all_files_sim_final[df_all_files_sim_final['file_name_tableros'].str.contains('BNTE')]
df_all_files_sim_final_ecom = df_all_files_sim_final[df_all_files_sim_final['file_name_tableros'].str.contains('ECOM')]

### PIVOT FECHA DE DEPOSITO

In [ ]:
pivot_fecha_deposito = pd.pivot_table(df_all_files_sim_final,index=['file_name_tableros','fecha_transaccion']
                                              ,columns=['fecha_deposito'],values=['monto']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_fecha_deposito.fillna(0,inplace=True)
pivot_fecha_deposito.head()

In [ ]:
pivot_fecha_deposito.shape

In [ ]:
writer = pd.ExcelWriter('ADQUIRIENTE NO CLIP JULIO 2021.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
pivot_fecha_deposito.to_excel(writer, sheet_name='RESUMEN')
df_all_files_sim_final_amex.to_excel(writer, sheet_name='AMEX', index=False)
df_all_files_sim_final_grrcn.to_excel(writer, sheet_name='GRRCN', index=False)
df_all_files_sim_final_bnte.to_excel(writer, sheet_name='BNTE', index=False)
df_all_files_sim_final_ecom.to_excel(writer, sheet_name='E-COMMERCE', index=False)
writer.save()
writer.close()